In [1]:
import os
import sys
import pandas as pd
import ast

# Import library with current code functions
sys.path.append(os.path.join("..", "lib"))
import video_adjuster_functions as vid_adj_fun, general_functions as gf, files_paths as fp

# Defining variables

In [2]:
# Scan the folder and save the list of csv
FILE_LIST_TO_NORMALIZE = gf.find_files_in_all_subdirectories([fp.DATASET_YT, fp.DATASET_LOCAL], fp.VD_FEATURES_L2)

# Filter witch videos has the 'VD_FEATURES_L2.CSV' file and delete then from the list
FILE_LIST_TO_NORMALIZE_FILTERED = [feature_file for feature_file in FILE_LIST_TO_NORMALIZE if not os.path.exists(os.path.join(os.path.dirname(feature_file), fp.VD_FEATURES_L3))]

## Normalize each frame of each video

In [3]:
features_start = '0'
features_end = '67'
normalization_parameter = [[0, 16], 400]

In [ ]:
landmark_smoother = vid_adj_fun.LandmarkSmoother()

for i, current_path_location in enumerate(FILE_LIST_TO_NORMALIZE):

    print(f"{i+1} of {len(FILE_LIST_TO_NORMALIZE)}: Starting to process {current_path_location}")

    landmarks_CSV = pd.read_csv(current_path_location)
    landmarks_0_to_66 = landmarks_CSV.loc[:, features_start:features_end]
    
    normalized_landmarks = []
    for _, row in landmarks_0_to_66.iterrows():
        landmarks_list = [list(ast.literal_eval(coord)) for coord in row]
        
        # Apply normalizations
        scale_factor, z_normalized = vid_adj_fun.z_normalization(landmarks_list, normalization_parameter[0], normalization_parameter[1])
        roll_normalized = vid_adj_fun.roll_normalization(z_normalized)
        normalized_landmarks.append(z_normalized)
    
    # Convert each landmark position into a tuple
    normalized_landmarks_to_tuple = [[tuple(inner) for inner in outer] for outer in normalized_landmarks]
    df_normalized_landmarks = pd.DataFrame(normalized_landmarks_to_tuple)
    
    # Concatenate left_columns with df_normalized_landmarks along the columns
    left_columns = landmarks_CSV.loc[:, 'video_id':'instance_ref']
    df_result = pd.concat([left_columns, df_normalized_landmarks], axis=1)

    algle_columns = landmarks_CSV.loc[:, 'rot_x':'rot_z']
    df_result = pd.concat([df_normalized_landmarks, algle_columns], axis=1)
    
    output_path = os.path.join(os.path.dirname(current_path_location), fp.VD_FEATURES_L3)

    df_result.to_csv(output_path, index=False)